In [24]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

In [25]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)

splitfolders.ratio(
    "E:\datasets\dataset-clear",
    output='C:\data\dataset\split',
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    group_prefix=None,
    move=False)


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256 , 256)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256 , 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset_noweather'
run["config/data_dir"] = data_dir

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-86


In [26]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 99
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)
                    run["logs/training/batch/loss"].append(loss)
                    run["logs/training/batch/acc"].append(acc)

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_loss:4f}')
    

    model.load_state_dict(best_model_wts)
    return model

In [27]:
model_ft = models.resnet152(pretrained=True) 
num_ftrs = model_ft.fc.in_features
# num_ftrs = model_ft.fc.in_features Для рес нета , но у эфинета другая архитектура 

model_ft.fc = nn.Linear(num_ftrs, len(class_names))
# cuda
model_ft = model_ft.to(device)
run["config/model"] = model_ft

criterion = nn.CrossEntropyLoss()
run["config/criterion"] = criterion

# оптимайзер  с лернинг рейт 
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
run["config/optimizer"] = optimizer_ft
run["parameters"] = model_ft.parameters()

# редактируем лернинг рейт каждые 7 шагов 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [28]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)
torch.save(model_ft,'resnet152-bestloss-no_weather.pth')
torch.save(model_ft.state_dict(), "weights-resnet152-bestloss-no_weather.pth")
run.stop()

Epoch 0/19
----------
train Loss: 1.3759 Acc: 0.5176
val Loss: 1.1270 Acc: 0.6212

Epoch 1/19
----------
train Loss: 0.9698 Acc: 0.6606
val Loss: 0.8102 Acc: 0.7403

Epoch 2/19
----------
train Loss: 0.8354 Acc: 0.7098
val Loss: 0.6670 Acc: 0.7816

Epoch 3/19
----------
train Loss: 0.7590 Acc: 0.7416
val Loss: 0.8302 Acc: 0.7201

Epoch 4/19
----------
train Loss: 0.7245 Acc: 0.7515
val Loss: 1.5089 Acc: 0.7416

Epoch 5/19
----------
train Loss: 0.7103 Acc: 0.7605
val Loss: 0.8775 Acc: 0.7812

Epoch 6/19
----------
train Loss: 0.6533 Acc: 0.7765
val Loss: 0.5613 Acc: 0.8074

Epoch 7/19
----------
train Loss: 0.4861 Acc: 0.8327
val Loss: 0.4164 Acc: 0.8594

Epoch 8/19
----------
train Loss: 0.4297 Acc: 0.8504
val Loss: 0.3909 Acc: 0.8697

Epoch 9/19
----------
train Loss: 0.4085 Acc: 0.8606
val Loss: 0.3875 Acc: 0.8732

Epoch 10/19
----------
train Loss: 0.3860 Acc: 0.8685
val Loss: 0.3797 Acc: 0.8654

Epoch 11/19
----------
train Loss: 0.3661 Acc: 0.8722
val Loss: 0.3657 Acc: 0.8758

Ep

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: logs/training/batch/loss. Invalid point: 2023-04-02T13:44:50.986Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: logs/training/batch/loss. Invalid point: 2023-04-02T13:44:51.584Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: logs/training/batch/loss. Invalid point: 2023-04-02T13:44:52.182Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: logs/training/batch/loss. Invalid point: 2023-04-02T13:44:52.780Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: logs/training/batch/loss. Invalid point: 2023-04-02T13:44:53.377Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute

Communication with Neptune restored!
train Loss: 0.2934 Acc: 0.8985
val Loss: 0.3463 Acc: 0.8822

Epoch 18/19
----------
train Loss: 0.2902 Acc: 0.9022
val Loss: 0.3558 Acc: 0.8779

Epoch 19/19
----------
train Loss: 0.2911 Acc: 0.9000
val Loss: 0.3463 Acc: 0.8826

Training complete in 181m 43s
Best val Loss: 0.338932
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
All 2 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/neas1231/Neas1231/e/NEAS-86/metadata


In [23]:
import gc
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
gc.collect()

19030